In [ ]:
import cv2
from astropy.io import fits
from glob import glob
import numpy as np
import os

# Ruta de la carpeta con los archivos FITS
folder_fits = r"C:\Users\eduar\OneDrive\Escritorio\SharpCap Captures\2024-03-11\M42\21_33_25"

# Obtener la lista de archivos FITS en la carpeta
fits_files = glob(os.path.join(folder_fits, "*.fits"))

# Variables para el video
video = None
height, width = None, None

# Iterar sobre cada archivo FITS
for fits_file in fits_files:
    # Abrir el archivo FITS
    hdul = fits.open(fits_file)

    # Obtener los datos de la imagen FITS
    fits_data = hdul[0].data.T

    # Escalar los valores de píxeles a 0-255
    fits_data = (
        (fits_data - np.min(fits_data)) / (np.max(fits_data) - np.min(fits_data)) * 255
    )

    # Convertir la imagen FITS a formato de imagen OpenCV (uint8)
    fits_data_uint8 = np.uint8(fits_data)

    # Obtener las dimensiones de la imagen
    height, width = fits_data_uint8.shape

    # Inicializar el video una vez que se obtengan las dimensiones
    if video is None:
        video = cv2.VideoWriter(
            "M42_Prueba.avi", cv2.VideoWriter_fourcc(*"DIVX"), 4, (width, height))

    # Añadir la imagen al video
    video.write(fits_data_uint8)

# Liberar el objeto VideoWriter
if video is not None:
    video.release()